In [38]:
import urllib
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

'''
    ['0', 'Отзывы', 'f=9', 'RENAULT LOGAN'],
    ['1', 'Двигатель', 'f=15', 'RENAULT LOGAN'],
    ['2', 'Трансмиссия', 'f=16', 'RENAULT LOGAN'],
    ['3', 'Ходовая', 'f=17', 'RENAULT LOGAN'],
    ['4', 'Колеса', 'f=105', 'RENAULT LOGAN'],
    ['5', 'Электрооборудование', 'f=18', 'RENAULT LOGAN'],
    ['6', 'Мультимедиа', 'f=23', 'RENAULT LOGAN'],
    ['7', 'Кузов', 'f=19', 'RENAULT LOGAN'],
    ['8', 'Жидкости', 'f=20', 'RENAULT LOGAN'],
    ['9', 'Сервис', 'f=13', 'RENAULT LOGAN'],
    ['10', 'Эксплуатация', 'f=24', 'RENAULT LOGAN'],
    ['11', 'Тюнинг', 'f=21', 'RENAULT LOGAN'],
    ['12', 'Кузов-Салон', 'f=104', 'RENAULT LOGAN'],
    ['13', 'Тюнинг-Допоборудование', 'f=22', 'RENAULT LOGAN']
'''

site = 'https://renaultlogan2.ru/forum/'
folder_names = [
    ['4', 'Колеса', 'f=105', 'RENAULT LOGAN'],
    ['5', 'Электрооборудование', 'f=18', 'RENAULT LOGAN'],
    ['6', 'Мультимедиа', 'f=23', 'RENAULT LOGAN'],
    ['7', 'Кузов', 'f=19', 'RENAULT LOGAN'],
    ['8', 'Жидкости', 'f=20', 'RENAULT LOGAN'],
    ['9', 'Сервис', 'f=13', 'RENAULT LOGAN'],
    ['10', 'Эксплуатация', 'f=24', 'RENAULT LOGAN'],
    ['11', 'Тюнинг', 'f=21', 'RENAULT LOGAN'],
    ['12', 'Кузов-Салон', 'f=104', 'RENAULT LOGAN'],
    ['13', 'Тюнинг-Допоборудование', 'f=22', 'RENAULT LOGAN']
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [28]:
def collect_themes(site, folder):
    
    site_address = site + 'forumdisplay.php?' + folder
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for link in soup.find_all('a'):
            if link.has_attr('href') and link.has_attr('id'):
                if re.search('thread_title_', str(link.get('id'))):
                    s = link.get('href')
                    s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                    themes.append([link.get_text(), s])
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [35]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
        
        print(theme[0])
        theme_address = theme[1]
        thread = re.search('t=[0-9]+', theme_address)[0]
        flag = True
        theme_messages = []
        post_data = {}
        author_data = {}        
        while flag:
            try:
                resp = urlopen(theme_address) # скачиваем файл
            except:
                break
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем некоторые метаданные сообщений на текущей странице
            tables = soup.find_all('table')
            for next_table in tables:
                meta = next_table.find_all('div')
                for next_meta in meta:
                    if next_meta.has_attr('class') and next_meta['class'] == ['panelbg']:
                        post_date = next_meta.get_text().strip()
                        post_data[re.sub('post', '', next_table['id'])] = [post_date]
                meta = next_table.find_all('li')
                for next_meta in meta:
                    if next_meta.has_attr('class') and next_meta['class'] == ['vbmenu_control']:
                        author_nickname = next_meta.get_text()
                        author = next_meta.find('a')
                        if author != None:
                            author_id = re.search('u=[0-9]+', author['href'])[0]
                        else:
                            author_id = 0
                        post_data[re.sub('post', '', next_table['id'])].append([author_id, author_nickname])
            #Собираем сами сообщения
            divs = soup.find_all('div')
            for div in divs:
                if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                    continue
                if div.has_attr('id') and re.search('post_message_', div['id']):
                    message_id = re.sub('post_message_', '', div['id'])
                    if message_id in post_data:
                        post_date = post_data[message_id][0]
                        author = post_data[message_id][1][1]
                    else:
                        post_date = ''
                        author = ''
                    for item in div.children:
                        if item.name == 'div':
                            item.clear()
                    s = re.sub('[\']+', '`', div.get_text())
                    s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                    if len(s) > 2:
                        theme_messages.append([post_date, author, s])
            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if not flag and link.has_attr('href') and link.has_attr('rel'):
                    if link.get('rel')[0] == 'next':
                        flag = True
                        s = link.get('href')
                        theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
        messages.append(['\''+ theme[0] + '\'', theme_messages])
    return messages

In [30]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Time,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [31]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)

In [6]:
%load_ext ipytelegram
%reload_ext ipytelegram
import telepot
bot = telepot.Bot('1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A')
response = bot.getUpdates()
%telegram_setup 1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A 1305740495

In [39]:
%%telegram_send Next message
scraper(site, folder_names)
'Информация собрана'

Докатка
Зимняя резина на новый Logan/Sandero - выбираем, обсуждаем
Летняя резина, что будем выбирать?
Колесные болты
Варианты штамповки
Сезонное "переобувание". Вопросы и проблемы.
Родные шины formula energy
Диски на 16 для нового Логана
О балансировке новых литых дисков
Какие диски подходят? - Размерность шин и дисков
Диски К&К - проектируют пионеры?
Чтобы НЕ прикипал колёсный диск к ступице
Неравномерный износ протектора задних шин
Направленность рисунка протектора
Секретки для колёс
Выбор литых дисков по производителю
хранение дисков и шин
Маленькие колпаки 6001548400 от первого Логана
Какие диски и шины идут с салона?
Диски на 17
Параметры шин и дисков
Нокиан для РЕНО



Лампы
Аккумуляторная батарея
Автоматическое отключение всего на стоянке
Стартер
Схема электрооборудования нового Логана
Лягушка педали тормоза
Замена повторителя бокового зеркала Рено Логан 2014 г.в.
Подключение к бортовой системе питания
Регулировка ближнего света фар
Внезапно перестал работать омыватель стекла.
Противотуманные фары на новый Логан 2
считыватель кодов (ошибок)
Проблемы с электроникой
Бортовой компьютер
Датчик кислородный
Глюк датчика уровня топлива или "мозгов"?
Неправильная работа дворников
Ошибка DF1058
Зеркальный элемент
6+ лет: профилактика проводки, чистка контактов и т.д.
После небольшого простоя не заводиться
А есть ли чип в ключе?
Не работает ближний свет.
Подсветка кнопок стеклоподъемников на дверях
Вполнакала горит кнопка обогрева заднего стекла
Подрулевой переключатель правый  стеклоочистителей с БК
Работа стеклоочистителя
Проблема с запуском мотора
Парктроник для Логана
Датчик температуры наружного воздуха в зеркале
Подсветка зоны высадки, подсветка дверей,

Качественная музыка
Штатная магнитола
Громкая связь
Media Nav
Магнитола на базе Android
Хрип СЧ динамика .
Выбор навигатора
Media Nav в  Renault LOGAN Stepway 2018 v ПО 1.0.13.1
Какой провод "+" а какой "-" на передних колонках
Фонит сабвуфер
Установка сабвуфера
Альтернатива Renault Media NAV
GPS Антенна Renault Media Nav
Обновление карт для навигации на новом Логане 2014
Антенна на крыше
Мороз и динамики
Medi@skin - бесплатная альтернативная прошивка для MediaNav
Menavrus-прошивка с несколькими навигациями, интернетом, видеоплеером, камерой и OBD
Блины 16х9 в заднюю полку.
Снятие облицовок двери+Штатная аудиосистема MediaNav
Как вставить 2din магнитолу?
Комплектация без музыки вообще, кто ставил?
Phantom 2210 wifi проблемы
Логан NEW 2014: есть ли аудиоподготовка в комплектации Confort (АвтоВАЗ)?
Проводка под задние колонки комплектация Access
Размер динамиков
Автовключение магнитолы при запуске двигателя
Коннектор ISO-ANT 2
Жгут подключения подрулевого пульта магнитолы
Не работают дин

Выпирает порог сзади. Кто-нибудь подобную проблему решал?
Шумоизоляция
Антикоррозийная обработка кузова нового Логан/Сандеро
Опрос: Ваш цвет кузова
Вибрация по кузову на скорости 85-90 км/ч
Возможно ли прикрыть петли капота на новом логане?
Защита кромки крыла
Хромированные детали на переднем бампере
Качество ЛКП (лакокрасочного покрытия) нового Logan и Sandero 2014
Жёсткость кузова Logan 2
Дополнительные уплотнители
Передний бампер
Странный оттенок у OV369
Заглушка снизу порога
Фрагментарный кузовной ремонт
Антикоррозийная обработка автомобиля
Задний бампер
Концевик багажника у Сандеро - где он?
Шкура дверей протирает краску
Крепление брызговиков
Что протекает под днищем?
Резиновые заглушки в двери
Дренажные отверстия кузова
Защита кромок дверей
Подкраска сколов
Кто знает как меняется шумоизоляция моторного отсека?
Крепкий ли у Логана кузов?! Да!
Ручки отбивают краску с двери
Шумоизоляция капота отваливается
ЛКП арки заднего колеса
Вода на блоке предохранителей
Полировка кузова
Защита

Моторное масло для Логана
Замена масла в КПП
Бензин для логана, и его расход
Масло для ГУР
Срок службы тормозной жидкости?
Отработка - применяем или выкидываем?!
Эмульсия в масле
Эксплуатационные жидкости: обобщённая информация
Антифриз, охлаждающая жидкость
Консистентные смазки - что выбрать?
Рено логан жрет масло
Фильтр в бачок стеклоомывателя
Триботехнические составы
Контрольная лампа давления масла
Стоимость газа по регионам.
Форсунки "омывайки"
Объем топливного бака
Как слить бензин с нового Логана?
Избыточное давление в бензобаке
Кто сколько проезжал на горящей лампочке остатка топлива?
Попадает ли бензин в крышку бензобака?
Замерзла "незамерзайка"
Сломал масляный щуп. Помогите...
Антифриз - доливка и полная замена.
Уходит тосол
Какой объем бачка омывателя лобового стекла?
Топливный фильтр



...

Каталожные номера деталей
Центральный замок
Дворники и омыватель лобового стекла
Государственный технический осмотр
Утепление двигателя зимой
Стоит ли брать автомобиль в жирной комплектации?
Стоимость владения Логаном-2
Приборная панель от Клио в Логан
Активация полезных опций
Список частых проблем при эксплутации автомобиля
Заглушки ламп в фаре
Руководство по эксплуатации нового Логана/Сандеро 2014
Крышка и пробка бензобака
Проблемы с показаниями уровня топлива
Штатный домкрат или ...?
Дубликат ключей
Машина бьётся.
Забрызгивает боковые стекла и зеркала.
Мороз! Что вы предпринимаете, чтобы авто нормально заводилось?
Коды ошибок
Бачок омывателя
Каталожные номера деталей
Упор капота
Торекс и др инструменты для авто -выбор.
Нужна помощь!
Незадокументированные различия между комплектациями
Свист при движении
Шум/потрескивание на 3й передаче на 2000 об в районе левого колеса
Заедает замок зажигания
Ароматизатор салона
Протяжка
Батарея ключа зажигания
Штатный балонник
Задний дворник сам вкл

Система контроля "слепых" зон
Нештатные Выпуск, Впуск двигателя.
Логан->Степвей
Омыватель фар
Держатель для мобильника
Защита радиатора
Рейлинги PTG для логана 2
Тонирование стекол
Ветровики (они же дефлекторы, дождевики)
Задние подкрылки
Спойлер-Обвесы для тюнинга
Козырёк на заднее стекло
Бампер от Степвея на Логан
Стоит ли обклеивать фары пленкой?
Альтернативные запчасти на новый логан?
Убрать надписи логан, рено?
Скоба регулировки положения передних сидений
Спойлер капота
Нужен размер надписи renault
Шторки на окна
Избавление от хрома на кузове.
Как правильно покрасить колпаки на диски.
Внутренние ручки передних дверей
Люк
Увеличить топливный бак
Защита салазок сидений
Пружины от степвэя
Рестайлинг рычага КПП
Хочу облагородить Логана.
Глазки на зеркала ;)
Яркая led подсветка номера авто.
МОЛДИНГИ ДВЕРЕЙ
Занижение нового Логана
Led освещение в логане
Защитная сетка в бампер своими руками
Детали в багажнике царапают/режут руки? Есть решение!
LED фара дополнительная для трассы.
Хромиро

Сиденья от Каптура в Рено Логан 2 + чехлы из экокожи
В ногах у водителя есть вот такая деталь. 10 000 км и она разваливается.
Лобовое стекло с обогревом
Работа климат-контроля.
Перевозил ли кто-нибудь в Логане Икеевский диван ФРИХЕТЭН?
Чем закрыть решётки радиатора на зиму?
Подогрев сидений
Чехлы на новый Logan/Sandero 2014
Органайзер в багажник!!!
Лобовое стекло
Салонный фильтр
Работа кондиционера
Сверчки в салоне
Защита ковролина салона
Багажник
Как снять заводскую обивку с передних сидений?
Очистка (дезинфекция) системы кондиционирования
Замена водительского сидения в Логан 2.
Конденсат от кондиционера
Багажник на новом Логане 2014
Белесый налет внутри  в центе нижней части лобового стекла
Ручка открывания багажника
Вибрация при увеличении оборотов вентилятора отопителя
Подлокотник
Эргономика водительского места
Обивка сидений!
Запотевают окна
Обзор источников скрипа и стука
Что за сверчок?
О ремнях безопасности
Обзор зеркал
Работа печки и отопление салона
Коврики в салон и багажник

Multitronics - альтернативный БК на Логан
Фаркоп на Логан 2
Механические противоугонки
Выбор сигнализации и защита от угона
Плавный розжиг панели приборов
"Непрограммная" нейтраль
Диагностический разъем
Вопросы по видеорегистратору
Парктроники вперёд
ELM 327
Установка сигнализации
А как насчет размыкателя цепи бензонансоса как противоугон?
Бесключевой обходчик IMMO
Камера заднего вида
Багажники, релинги, крепления для велосипедов
Защита "сейф" ЭБУ
Датчик дождя.
Обратная связь без сигнализации
Пользуетесь ли вы автозапуском?
Видеонаблюдение - пассивная защита автомобиля и не только.
Светодиодный проектор логотипа (подсветка при открытых дверях)
Подогрев двигателя.
Замена кулисы КПП Easy'R
Ксенон для  Рено Логан 2.
Чем закрыть личинку дверного замка
"Перехват-универсал"
Тест автосигнализаций 2016
Перенести диагностический разъём
StarLine E-91
Гарант Блок
Гарант Panzer
Установка замка капота Starline L12
Тест штатной сигнализации автомобиля
USB-зарядка на 2-4 устройства
Защита колесных ар

'Информация собрана'

In [ ]:
import pandas as pd
mess = pd.read_csv('./LADA VESTA/LADA VESTA_10_Эксплуатация.csv', quotechar="'")
mess

In [14]:
folder_names = [['13', 'Тюнинг-Допоборудование', 'f=22', 'RENAULT LOGAN']]
folder = folder_names[0][2]
current_themes = collect_themes(site, folder)

In [ ]:
current_themes

In [ ]:
themes = []
flag = True

while flag:
    
    resp = urlopen(site_address) # скачиваем файл
    html = resp.read().decode('utf-8') # считываем содержимое
    soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
    flag = False
    for link in soup.find_all('a'):
        if link.has_attr('href') and link.has_attr('id'):
            if re.search('thread_title_', str(link.get('id'))):
                s = link.get('href')
                s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                themes.append([link.get_text(), s])
        if not flag and link.has_attr('href') and link.has_attr('rel'):
            if link.get('rel')[0] == 'next':
                flag = True
                s = link.get('href')
                site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]

In [24]:
themes = [['current_themes',
  'https://renaultlogan2.ru/forum/showthread.php?t=4368']]

In [25]:
messages = []
for theme in tqdm(themes):
    theme_address = theme[1]
    thread = re.search('t=[0-9]+', theme_address)[0]
    flag = True
    theme_messages = []
    post_data = {}
    author_data = {}
    while flag:
        resp = urlopen(theme_address) # скачиваем файл
        html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
        #Собираем некоторые метаданные сообщений на текущей странице
        tables = soup.find_all('table')
        for next_table in tables:
            if next_table.has_attr('id') and re.search('post[0-9]+', next_table['id']):
                meta = next_table.find_all('div')
                for next_meta in meta:
                    if next_meta.has_attr('class') and next_meta['class'] == ['panelbg']:
                        post_date = next_meta.get_text().strip()
                        post_data[re.sub('post', '', next_table['id'])] = [post_date]
                meta = next_table.find_all('li')
                for next_meta in meta:
                    if next_meta.has_attr('class') and next_meta['class'] == ['vbmenu_control']:
                        author_nickname = next_meta.get_text()
                        author = next_meta.find('a')
                        if author != None:
                            author_id = re.search('u=[0-9]+', author['href'])[0]
                        else:
                            author_id = 0
                        post_data[re.sub('post', '', next_table['id'])].append([author_id, author_nickname])
        #Собираем сами сообщения
        divs = soup.find_all('div')
        for div in divs:
            if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                continue
            if div.has_attr('id') and re.search('post_message_', div['id']):
                message_id = re.sub('post_message_', '', div['id'])
                if message_id in post_data:
                    post_date = post_data[message_id][0]
                    author = post_data[message_id][1][1]
                else:
                    post_date = ''
                    author = ''
                for item in div.children:
                    if item.name == 'div':
                        item.clear()
                s = re.sub('[\']+', '`', div.get_text())
                s = '\'' + re.sub('[\s]+', ' ', s) + '\''
                if len(s) > 0:
                    theme_messages.append([post_date, author, s])
        # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
        # Если ссылка найдена, 
        flag = False 
        for link in soup.find_all('a'):
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
    messages.append(['\''+ theme[0] + '\'', theme_messages])

In [26]:
messages

[["'current_themes'",
  [['03.12.2014, 19:32',
    'КСН',
    "' хочу поставеть механическою противоугонку.Кто что посаветует.НЕ очень дорагую. '"],
   ['03.12.2014, 19:37',
    'nik797',
    "' Гарант на рулевой вал. По отзывам самое лучшее на данный момент. '"],
   ['03.12.2014, 19:39',
    'mug08',
    '\' Только со временем ленятся его ставить "выходя из авто на 5 минут" \''],
   ['03.12.2014, 19:57', 'КСН', "' а что небуть на каробку '"],
   ['03.12.2014, 19:57',
    'wolf',
    "' Из новых Перехват и Рыцарь правда не дёшевы. '"],
   ['03.12.2014, 19:59', 'wolf', "' Просто набери в поиске там всё выдаст. '"],
   ['03.12.2014, 20:00',
    'nik797',
    '\' Это точно. Когда-то стоял такой на пятнашке. Замыкал только если на долго оставлял машину. А угонщику достаточно пары минут.)) Но этой бедой страдают все мех.устройства. Что "кочерга" на руль, что на запор на рычаг КПП.)) \''],
   ['03.12.2014, 20:03',
    'wolf',
    "' То что выше написал они вроде по надёжней будут. '"],
   ['

In [85]:
f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
with open(f_name, 'w', encoding='utf-8') as ouf:
    ouf.write('Code,Folder,Theme,Date,Author,Message\n')
    for message in messages:
        header = folder_name[:2] + [message[0]]
        for item in message[1]:
            ouf.write(','.join(header + item))
            ouf.write('\n')

In [69]:
folder_name = ['0', 'Отзывы', 'f=25', 'LADA VESTA']
f_name = folder_name[3] + '/' + folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv'
f_name

'LADA VESTA/LADA VESTA_0_Отзывы.csv'

In [ ]:
'''messages = []
theme_address = 'http://www.lada-vesta.net/showthread.php?t=9571'
resp = urlopen(theme_address) # скачиваем файл
html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
soup = BeautifulSoup(html, 'html.parser') # делаем суп
divs = soup.find_all('div')
for div in divs:
    if div.has_attr('id'):
        if re.search('post_message_', div['id']):
            for item in div.children:
                if item.name != 'div':
                    s = str(item).strip()
                    if len(s) > 0:
                        messages.append([s])
print(messages)'''